**第三周作业 说明**

**1. SVD应用于菜肴推荐**（参考《机器学习实战》14.5 ）

可参考 https://github.com/apachecn/AiLearning/blob/master/src/py2.x/ml/14.SVD/svdRecommend.py

菜肴数据如下，矩阵第$i$行第$j$列的数值代表用户$i$对于菜肴$j$的评分，如果用户没有尝过某道菜，则评级为0。

```
[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
[0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
[0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
[3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
[5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
[0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
[4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
[0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
[0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
[0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
[1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]
```

我们希望构建一个推荐系统：给定一个用户，系统会为此用户返回N个最好的推荐菜。其实现流程大致如下：

（1）寻找用户没有评级的菜肴，即在用户-物品矩阵中的0值。

（2）在用户没有评级的所有物品中，对每个物品预计一个可能的评级分数。

（3）对这些物品的评分从高到低进行排序，返回前N个物品。

**任务：**对于上述的菜肴数据，基于物品相似度构建推荐引擎，为用户4和用户9推荐两个菜肴；并用SVD提高推荐的效果。

In [1]:
from numpy import linalg as la
from numpy import *

In [2]:
myMat = mat([[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])

#### 相似度计算函数
假定inA和inB 都是列向量，有三种距离计算方式：
+ 欧式距离
+ 皮尔森相关系数
+ 余弦相似度

In [3]:
# 欧氏距离
def ecludSim(inA, inB):
    return 1.0/(1.0 + la.norm(inA - inB))

# 皮尔森相关系数
def pearsSim(inA, inB):
    # 如果不存在，该函数返回1.0，此时两个向量完全相关。
    if len(inA) < 3:
        return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar=0)[0][1]

## 余弦相似度
def cosSim(inA, inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5*(num/denom)

#### 计算用户对未评分物品的估计评分值


对于用户评分过的每个物品进行遍历，并将它和其他物品比较，根据两个物品的已被评分的重合元素计算相似度，例如计算[0 0 0 4 5 0 4 0 0 0 0]和[0 0 0 2 5 0 5 0 0 0 2]的相似度，即计算[4.5,4]和[2,5,5]的相似度，如果两者没有任何重合元素，则相似度为0且中止本次循环。<br>
用户user对物品item的评分估计值就等于所有这些评分的物品的加权评分和，其中权重是物品相似度。

In [4]:
# 基于物品相似度的推荐引擎
def standEst(dataMat, user, simMeas, item):
    """standEst(计算某用户未评分物品中，以对该物品和其他物品评分的用户的物品相似度，然后进行综合评分)
    Args:
        dataMat         训练数据集
        user            用户编号
        simMeas         相似度计算方法
        item            未评分的物品编号
    Returns:
        ratSimTotal/simTotal     评分（0～5之间的值）
    """
    # 得到数据集中的物品数目
    n = shape(dataMat)[1]
    # 初始化两个评分值
    simTotal = 0.0
    ratSimTotal = 0.0
    # 遍历行中的每个物品（对用户评过分的物品进行遍历，并将它与其他物品进行比较）
    for j in range(n):
        userRating = dataMat[user, j]
        # 如果某个物品的评分值为0，则跳过这个物品
        if userRating == 0:
            continue
        # 寻找两个用户都评级的物品
        # 变量 overLap 给出的是两个物品当中已经被评分的那个元素的索引ID
        # logical_and 计算x1和x2元素的真值。
        overLap = nonzero(logical_and(dataMat[:, item].A > 0, dataMat[:, j].A > 0))[0]
        # 如果相似度为0，则两着没有任何重合元素，终止本次循环
        if len(overLap) == 0:
            similarity = 0
        # 如果存在重合的物品，则基于这些重合物重新计算相似度。
        else:
            similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        # print 'the %d and %d similarity is : %f'(iten,j,similarity)
        # 相似度会不断累加，每次计算时还考虑相似度和当前用户评分的乘积
        # similarity  用户相似度，   userRating 用户评分
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    # 通过除以所有的评分总和，对上述相似度评分的乘积进行归一化
    else:
        return ratSimTotal/simTotal

#### 推荐系统

+ 推荐系统首先对给定的用户建立一个未评分的物品列表。<br>
+ 如果不存在未评分物品，那么就退出函数；否则，在所有的未评分物品上进行循环。<br>
+ 对每个未评分物品，则通过调用standEst()来产生该物品的预测得分。该物品的编号和估计得分值会放在一个元素列表itemScores中。
+ 最后按照估计得分，对该列表进行排序并返回

In [5]:
# recommend()函数，就是推荐引擎，它默认调用standEst()函数，产生了最高的N个推荐结果。
# 如果不指定N的大小，则默认值为3。该函数另外的参数还包括相似度计算方法和估计方法
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    """svdEst( )
    Args:
        dataMat         训练数据集
        user            用户编号
        simMeas         相似度计算方法
        estMethod       使用的推荐算法
    Returns:
        返回最终 N 个推荐结果
    """
    # 寻找未评级的物品
    # 对给定的用户建立一个未评分的物品列表
    unratedItems = nonzero(dataMat[user, :].A == 0)[1]
    # 如果不存在未评分物品，那么就退出函数
    if len(unratedItems) == 0:
        return 'you rated everything'
    # 物品的编号和评分值
    itemScores = []
    # 在未评分物品上进行循环
    for item in unratedItems:
        # 获取 item 该物品的评分
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    # 按照评分得分 进行逆排序，获取前N个未评级物品进行推荐
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[: N]

#### 默认推荐效果
+ 用户4的推荐结果

In [6]:
print(recommend(myMat, 3, 2,simMeas=ecludSim)) #欧式距离
print(recommend(myMat, 3, 2,simMeas=pearsSim)) #皮尔森相关系数
print(recommend(myMat, 3, 2,simMeas=cosSim)) #余弦相似度

[(10, 2.8783783783783785), (4, 2.0)]
[(10, 2.8), (4, 2.0)]
[(10, 2.8), (4, 2.0)]


+ 用户9的推荐结果

In [7]:
print(recommend(myMat, 8, 2,simMeas=ecludSim)) #欧式距离
print(recommend(myMat, 8, 2,simMeas=pearsSim)) #皮尔森相关系数
print(recommend(myMat, 8, 2,simMeas=cosSim)) #余弦相似度

[(4, 2.142857142857143), (0, 2.0)]
[(4, 3.2857142857142856), (0, 2.0)]
[(4, 3.0027358160539563), (0, 2.0)]


使用默认推荐引擎，距离的计算方法的不同并不会对推荐结果产生很大的影响，

为用户4推荐11和3两个菜品，为用户9推荐5和1两个菜品。

#### 利用 SVD 提高推荐的效果

利用SVD将矩阵转化到低维空间中计算相似度。


In [8]:
U, Sigma, VT = la.svd(myMat)
Sig2 = Sigma**2
print(sum(Sig2[:2])/sum(Sig2))
print(sum(Sig2[:3])/sum(Sig2))

0.6989475267737237
0.9234322677630622


前三个元素所包含的能量大于90%。所以将一个原矩阵转换成一个3维的矩阵。对转换后的三维空间构造相似度计算函数svdEst。

In [9]:
Sig3 = mat(eye(3) * Sigma[: 3])
xformedItems = myMat.T * U[:, :3] * Sig3.I
xformedItems

matrix([[-0.45137416,  0.03084799, -0.00290108],
        [-0.36239706,  0.02584428, -0.00189127],
        [-0.46879252,  0.03296133, -0.00281253],
        [-0.01007685, -0.34024331, -0.22728592],
        [-0.01567036, -0.38750193,  0.61197998],
        [-0.01664563, -0.52000097, -0.3608907 ],
        [-0.00474684, -0.18887149, -0.00924222],
        [-0.46712774,  0.00389831,  0.03349951],
        [-0.47223188,  0.02853952, -0.00504059],
        [-0.01591788, -0.39205093,  0.55707516],
        [-0.0552444 , -0.52034959, -0.36330956]])

In [10]:
def svdEst(dataMat, user, simMeas, item):
    """svdEst( )
    Args:
        dataMat         训练数据集
        user            用户编号
        simMeas         相似度计算方法
        item            未评分的物品编号
    Returns:
        ratSimTotal/simTotal     评分（0～5之间的值）
    """
    n = shape(dataMat)[1]

    simTotal = 0.0
    ratSimTotal = 0.0
    U, Sigma, VT = la.svd(dataMat)

    # 如果要进行矩阵运算，就必须要用这些奇异值构建出一个对角矩阵
    Sig3 = mat(eye(3) * Sigma[: 3])
    # 利用U矩阵将物品转换到低维空间中，构建转换后的物品(物品+3个主要的特征)
    xformedItems = dataMat.T * U[:, :3] * Sig3.I

    # 对于给定的用户，for循环在用户对应行的元素上进行遍历
    # 这和standEst()函数中的for循环的目的一样，只不过这里的相似度计算时在低维空间下进行的。
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0 or j == item:
            continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        # 计算估计评分
        return ratSimTotal/simTotal

#### SVD推荐引擎推荐效果

In [11]:
print(recommend(myMat, 3, 2,simMeas=ecludSim,estMethod=svdEst)) #欧式距离
print(recommend(myMat, 3, 2,simMeas=pearsSim,estMethod=svdEst)) #皮尔森相关系数
print(recommend(myMat, 3, 2,simMeas=cosSim,estMethod=svdEst)) #余弦相似度

[(6, 2.8020413196212686), (3, 2.8018347726404214)]
[(4, 2.7827858300715773), (9, 2.782287686525049)]
[(10, 2.797396529835738), (5, 2.797183757137651)]


In [12]:
print(recommend(myMat, 8, 2,simMeas=ecludSim,estMethod=svdEst)) #欧式距离
print(recommend(myMat, 8, 2,simMeas=pearsSim,estMethod=svdEst)) #皮尔森相关系数
print(recommend(myMat, 8, 2,simMeas=cosSim,estMethod=svdEst)) #余弦相似度

[(1, 2.5071588465128785), (0, 2.491990377026008)]
[(4, 2.4521567423719515), (7, 2.2143128739617004)]
[(4, 2.388669784821897), (8, 2.386959260544294)]
